# Wprowadzenie do Databricks Lakehouse

**Cel szkoleniowy:** Zrozumienie koncepcji Lakehouse, poznanie podstawowych elementów platformy Databricks oraz konfiguracji środowiska Unity Catalog.

**Zakres tematyczny:**
- Koncepcja Lakehouse (Data Lake + Data Warehouse)
- Elementy platformy: Workspace, Catalog Explorer, Repos, Volumes, DBFS
- Compute: clusters, autoscaling, spot instances, Photon Engine
- Notebooks: magic commands (%sql, %python, %md)
- Unity Catalog overview: katalogi, schematy, tabele
- Różnice między Hive Metastore a Unity Catalog

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z 2-4 workers
- **Czas trwania**: 20 minut

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie ewolucji architektur danych i miejsca Lakehouse w tym kontekście.

**Podstawowe pojęcia:**
- **Data Lake**: Scentralizowane repozytorium przechowujące surowe dane w różnych formatach (strukturalne, semi-strukturalne, niestrukturalne)
- **Data Warehouse**: Zoptymalizowane repozytorium danych strukturalnych do analityki biznesowej i raportowania
- **Lakehouse**: Nowoczesna architektura łącząca elastyczność Data Lake z niezawodnością i wydajnością Data Warehouse

**Dlaczego to ważne?**
Lakehouse eliminuje potrzebę utrzymywania dwóch oddzielnych systemów (Data Lake + Data Warehouse), redukując koszty, złożoność i opóźnienia w dostępie do danych. Dzięki Delta Lake uzyskujemy transakcyjność ACID, wersjonowanie danych i optymalizację zapytań bezpośrednio na plikach w Data Lake.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
import re

# Wyświetl kontekst użytkownika (zmienne z 00_setup)
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

In [0]:
# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

## Sekcja 1: Koncepcja Lakehouse Architecture

**Wprowadzenie teoretyczne:**

Lakehouse to nowoczesna architektura danych, która łączy zalety Data Lake (niski koszt przechowywania, wsparcie dla różnych formatów) z zaletami Data Warehouse (niezawodność, wydajność zapytań SQL, zarządzanie transakcjami). Kluczowym elementem jest Delta Lake - warstwa metadanych zapewniająca transakcyjność ACID na plikach Parquet.

**Kluczowe pojęcia:**
- **ACID Transactions**: Atomowość, Spójność, Izolacja, Trwałość - gwarancje zapewniające niezawodność operacji na danych
- **Delta Lake**: Open-source storage layer zapewniający transakcyjność na plikach w Data Lake
- **Unity Catalog**: Zunifikowany system zarządzania danymi, metadanymi i kontrolą dostępu

**Zastosowanie praktyczne:**
- Eliminacja duplikacji danych między systemami analitycznymi i operacyjnymi
- Jednoczesne wsparcie dla BI, Data Science i Machine Learning
- Redukcja kosztów infrastruktury i utrzymania

Porównanie tradycyjnej architektury vs Lakehouse

**Cel:** Wizualizacja różnic między tradycyjnym podejściem (Data Lake + Data Warehouse) a Lakehouse.

**Tradycyjna architektura:**
```
Raw Data → Data Lake (S3/ADLS) → ETL Process → Data Warehouse (Snowflake/Redshift) → BI Tools
                                ↓
                         ML/Data Science (separate copy)
```

**Lakehouse architektura:**
```
Raw Data → Delta Lake (single source of truth) → BI Tools + ML + Real-time Analytics
```

**Korzyści Lakehouse:**
- Jedna kopia danych (single source of truth)
- Niższe koszty przechowywania
- Eliminacja opóźnień synchronizacji
- Wspólne governance dla wszystkich use cases

## Sekcja 2: Elementy platformy Databricks

**Wprowadzenie teoretyczne:**

Platforma Databricks składa się z kilku kluczowych komponentów, które razem tworzą kompletne środowisko do pracy z danymi w architekturze Lakehouse.

**Kluczowe komponenty:**
- **Workspace**: Środowisko pracy zawierające notebooks, eksperymenty, foldery i zasoby
- **Catalog Explorer**: Interfejs do zarządzania katalogami, schematami, tabelami i widokami
- **Git Folders (dawniej Repos)**: Integracja z Git do wersjonowania notebooków i kodu
- **Volumes**: Zarządzanie plikami niestrukturalnymi (obrazy, modele, artifacts)
- **DBFS (Databricks File System)**: Wirtualny system plików nad cloud storage

**Zastosowanie praktyczne:**
- Workspace organizuje projekty i współpracę zespołową
- Catalog Explorer umożliwia eksplorację i governance danych
- Git Folders integruje development workflow z Git

### Przykład 2.1: Eksploracja Workspace

**Cel:** Zapoznanie się z interfejsem Databricks Workspace

**Elementy Workspace:**
1. **Sidebar** (lewa strona):
   - Workspace: Foldery i notebooki
   - Git Folders: Integracja Git
   - Compute: Zarządzanie klastrami
   - Workflows: Databricks Jobs
   - Catalog: Unity Catalog explorer

2. **Główny panel**: Edytor notebooków lub widok szczegółów

3. **Górna belka**: Szybki dostęp do compute, account, help

**Instrukcje nawigacji:**
- Użyj lewego menu do przełączania między sekcjami
- W sekcji Catalog możesz przeglądać katalogi, schematy i tabele
- W sekcji Compute zarządzasz klastrami Spark

### Przykład 2.2: Catalog Explorer - struktura Unity Catalog

**Cel:** Zrozumienie hierarchii obiektów w Unity Catalog

In [0]:
# Wyświetl aktualny katalog i schemat
current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
current_schema = spark.sql("SELECT current_schema()").collect()[0][0]

print(f"Aktualny katalog: {current_catalog}")
print(f"Aktualny schemat: {current_schema}")

**Hierarchia Unity Catalog:**

```
Metastore
  ├── Catalog (np. main, dev, prod)
  │   ├── Schema/Database (np. bronze, silver, gold)
  │   │   ├── Tables (Delta Tables)
  │   │   ├── Views (SQL Views)
  │   │   ├── Functions (UDFs)
  │   │   └── Volumes (dla plików)
```

Unity Catalog organizuje dane w trzech poziomach: `catalog.schema.table`

**Wyjaśnienie:**

Unity Catalog organizuje dane w hierarchii: Metastore → Catalog → Schema → Objects (Tables/Views/Functions). Ta struktura umożliwia:
- Logiczne oddzielenie środowisk (dev/test/prod)
- Granularną kontrolę dostępu na każdym poziomie
- Łatwe zarządzanie namespace'ami i izolacją projektów

### Przykład 2.3: Przeglądanie katalogów i schematów

**Cel:** Programowe listowanie obiektów w Unity Catalog

In [0]:
# Lista wszystkich katalogów dostępnych dla użytkownika
catalogs_df = spark.sql("SHOW CATALOGS")
display(catalogs_df)

In [0]:
# Lista schematów w aktualnym katalogu
schemas_df = spark.sql(f"SHOW SCHEMAS IN {CATALOG}")
display(schemas_df)

**Wyjaśnienie:**

Polecenia `SHOW CATALOGS` i `SHOW SCHEMAS` pozwalają na eksplorację struktury Unity Catalog. Każdy użytkownik widzi tylko te obiekty, do których ma uprawnienia. Per-user izolacja (jak w naszym `00_setup`) zapewnia, że każdy uczestnik szkolenia ma własną przestrzeń roboczą.

## 2.4 Git Folders (Repos) i integracja z Git 

W praktyce praca z kodem w Databricks powinna opierać się o **Git Folders** (dawniej Repos), a nie o pojedyncze, osierocone notebooki w Workspace.

Typowy workflow:

1. **Utwórz Git Folder** w Databricks: `Workspace → Git Folders → Add Repo`.
2. **Połącz z Git** (GitHub / Azure DevOps / inne).
3. Pracuj na **branchach feature** (np. `feature/cleaning-module`).
4. Regularnie:
   - commituj i pushuj zmiany z Databricks do zdalnego repo,
   - twórz PR i merguj do main/dev.

Dobre praktyki:

- Jeden repo per projekt/domenę (np. `databricks-dea-training-kion`).
- Nie pracujemy w **Workspace root** – zawsze w **Git Folders**.
- Notebooki szkoleniowe, dane testowe i README mogą być w jednym repo.


## 2.5 Volumes vs DBFS – gdzie trzymać pliki?

W nowych workspace'ach opartych na Unity Catalog preferowanym miejscem przechowywania plików są **Volumes**.

- `dbfs:/` traktujemy jako warstwę **legacy** lub obszar pomocniczy.
- `volume://catalog.schema.volume_name` to w pełni zarządzany, kontrolowany przez UC obszar danych (uprawnienia, audit, lineage).

Przykład definicji Volume (SQL):

```sql
CREATE VOLUME IF NOT EXISTS ${catalog}.${schema}.training_volume
COMMENT 'Obszar roboczy na potrzeby szkoleń';
```

Przykład użycia w PySpark:

```python
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

volume_path = f"volume://{catalog}.{schema}.training_volume"
display(dbutils.fs.ls(volume_path))
```



## 2.6 Serverless SQL / SQL Warehouse – kiedy zamiast clustra notebookowego?

Oprócz klastrów notebookowych Databricks oferuje **SQL Warehouse (serverless)** – silnik zapytań SQL zoptymalizowany pod BI i analitykę ad‑hoc.

Kiedy używać:
- Raportowanie w Power BI / innych narzędziach BI.
- Analitycy biznesowi / power users, którzy pracują głównie w SQL.
- Interaktywne dashboardy i zapytania ad‑hoc do warstwy **Gold**.

Różnice do all‑purpose cluster:
- Rozliczanie w oparciu o **DBU SQL** (inne stawki).
- Automatyczny provisioning / scaling.
- Izolacja obciążenia BI od klastrów inżynierskich.


## Sekcja 3: Compute - Klastry Spark i Serverless

**Wprowadzenie teoretyczne:**

Klastry Spark w Databricks są środowiskiem wykonawczym dla przetwarzania danych. Tradycyjnie zarządzamy klastrami (All-Purpose, Job), ale platforma coraz mocniej ewoluuje w stronę **Serverless Compute**, gdzie infrastruktura jest w pełni zarządzana przez Databricks, startuje natychmiastowo i skaluje się automatycznie.

**Kluczowe pojęcia:**
- **All-Purpose Cluster**: Interaktywne klastry do analizy i rozwoju w notebookach.
- **Job Cluster**: Efemeryczne klastry dla automatyzowanych zadań.
- **Serverless Compute**: Bezobsługowa warstwa obliczeniowa (dostępna dla SQL Warehouses, Jobs, a coraz częściej dla Notebooków).
- **Autoscaling**: Automatyczne skalowanie liczby workerów.
- **Photon Engine**: Natywny engine wykonawczy w C++ przyspieszający zapytania.

**Zastosowanie praktyczne:**
- Serverless eliminuje czas oczekiwania na start klastra (instant startup).
- Autoscaling redukuje koszty przy zmiennym obciążeniu.
- Photon przyspiesza zapytania agregacyjne nawet 3x.

### Przykład 3.1: Informacje o klastrze

**Cel:** Sprawdzenie konfiguracji aktualnego klastra

In [0]:
# Informacje o Spark Context
spark_version = spark.version
app_name = spark.sparkContext.appName
master = spark.sparkContext.master

**Konfiguracja klastra:**

Podstawowe informacje o środowisku Spark i Databricks Runtime.

In [0]:
print(f"Spark Version: {spark_version}")
print(f"Aplikacja: {app_name}")
print(f"Master: {master}")

In [0]:
# Liczba executorów
num_executors = len(spark.sparkContext._jsc.sc().statusTracker().getExecutorInfos()) 

In [0]:
# Runtime version
dbr_version = spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion", "unknown")

In [0]:
# Photon włączony?
photon_enabled = spark.conf.get("spark.databricks.photon.enabled", "false")

**Wyjaśnienie:**

Ten kod pokazuje podstawowe informacje o klastrze Spark. Liczba executorów (workers) może się zmieniać dynamicznie przy włączonym autoscalingu. Photon Engine, jeśli włączony, automatycznie przyspiesza zapytania SQL i operacje DataFrame bez zmian w kodzie.

### Przykład 3.2: Best practices dla konfiguracji klastrów

**Cel:** Poznanie rekomendacji dla różnych use cases

**Dla Development (All-Purpose Cluster):**
- Runtime: 14.3 LTS (Long Term Support)
- Workers: 2-4 (autoscaling 2-8 dla większych projektów)
- Node type: Standard DS3_v2 (Azure) lub m5.xlarge (AWS)
- Photon: Włączony
- Spot instances: Nie (dla stabilności)

**Dla Production (Job Cluster):**
- Runtime: 14.3 LTS
- Workers: autoscaling 2-20 (zależnie od obciążenia)
- Node type: Memory-optimized (DS4_v2, m5.2xlarge)
- Photon: Włączony
- Spot instances: Tak (60-80% workers)
- Auto-termination: 10 minut nieaktywności

**Dla ML Workloads:**
- Runtime: 14.3 ML (zawiera biblioteki ML)
- Workers: GPU-enabled (NC6s_v3, p3.2xlarge)
- Single-node mode dla prototypowania

## Sekcja 4: Magic Commands w Notebookach

**Wprowadzenie teoretyczne:**

Databricks notebooks obsługują magic commands - specjalne polecenia zaczynające się od `%`, które kontrolują język komórki lub wykonują operacje systemowe. Magic commands umożliwiają mieszanie języków w jednym notebooku oraz interakcję z systemem plików.

**Dostępne magic commands:**
- **%python**: Komórka Python (domyślny)
- **%sql**: Komórka SQL
- **%scala**: Komórka Scala
- **%r**: Komórka R
- **%md**: Komórka Markdown (dokumentacja)
- **%fs**: Operacje na systemie plików (DBFS)
- **%sh**: Polecenia shell
- **%run**: Uruchomienie innego notebooka (jak import)
- **%pip**: Instalacja bibliotek Python (notebook-scoped)
- **%skip**: Skipuje komorke

**Zastosowanie praktyczne:**
- Łączenie SQL i Python w jednym workflow
- Dokumentacja inline z Markdown
- Operacje na plikach z %fs
- Modularyzacja kodu z %run
- Zarządzanie zależnościami z %pip


## Monitoring i logowanie – gdzie szukać problemów?

Przy pracy z klastrami i Jobami warto znać podstawowe miejsca, gdzie szukamy informacji diagnostycznych:

- **Cluster → Event log** – start/stop clustra, autoscaling, błędy infrastruktury.
- **Spark UI** (zakładki Jobs, SQL, Storage, Environment) – plan wykonania, shufflowanie, błędy na poziomie zadań.
- **Driver / Executor logs** – szczegółowe stacktrace'y wyjątków Pythona/Scali.
- **Job Run page** – status poszczególnych tasków, retry, czas wykonania.

Dobre praktyki:
- Przy dłuższych pipeline'ach zawsze sprawdzaj Spark UI (sekcja SQL/Jobs).
- Krytyczne logi aplikacyjne zapisuj do tabel Delta / storage, a nie tylko do logów clustra.


### Przykład 4.1: Demonstracja SQL magic command

**Cel:** Wykonanie zapytania SQL bezpośrednio w notebooku

In [0]:
%sql
-- SQL magic command pozwala pisać czyste SQL bez otoczki Pythona

SELECT 
  current_catalog() as catalog,
  current_schema() as schema,
  current_user() as user,
  current_timestamp() as timestamp

**Wyjaśnienie:**

Magic command `%sql` zmienia język komórki na SQL. Wyniki są automatycznie wyświetlane jako tabela. SQL w Databricks to pełny Spark SQL z rozszerzeniami Delta Lake.

### Przykład 4.2: File System operations z fs

**Cel:** Eksploracja systemu plików DBFS

In [0]:
# Lista katalogów głównych w DBFS
dbutils.fs.ls("/")

**Wyjaśnienie:**

DBFS (Databricks File System) to abstrakcja nad cloud storage (S3, ADLS, GCS). Komenda `%fs` lub `dbutils.fs` pozwala na operacje na plikach. W Unity Catalog zaleca się używanie **Volumes** zamiast DBFS dla lepszego governance.

### Przykład 4.3: Mieszanie języków - Python i SQL

**Cel:** Demonstracja płynnego przechodzenia między Python i SQL

In [0]:
# Python: Definicja danych surowych
data = [
    (1, "Alice", "Engineering", 95000),
    (2, "Bob", "Sales", 75000),
    (3, "Charlie", "Engineering", 105000),
    (4, "Diana", "Marketing", 68000),
    (5, "Eve", "Engineering", 98000)
]

# Definicja schematu
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("department", StringType(), False),
    StructField("salary", IntegerType(), False)
])

In [0]:
# Utworzenie DataFrame
df = spark.createDataFrame(data, schema)
display(df)

In [0]:
# Rejestracja jako temp view dla dostępu z SQL
df.createOrReplaceTempView("employees_temp")

**Temp view utworzony: employees_temp**

Temp view pozwala na dostęp do DataFrame z komórek SQL w tym samym notebooku.

In [0]:
%sql
-- SQL: Agregacja na danych z Python

SELECT 
  department,
  COUNT(*) as employee_count,
  AVG(salary) as avg_salary,
  MAX(salary) as max_salary
FROM employees_temp
GROUP BY department
ORDER BY avg_salary DESC

**Wyjaśnienie:**

Ten przykład pokazuje siłę notebooków Databricks: przygotowanie danych w Python (wygodne API, biblioteki), następnie analiza w SQL (deklaratywne zapytania, przejrzystość). Temp views są widoczne w całym notebooku niezależnie od języka komórki.

### Przykład 4.4: Zarządzanie bibliotekami (%pip)

W Databricks możemy instalować biblioteki Python specyficzne dla danego notebooka (notebook-scoped libraries) używając komendy `%pip`. Jest to zalecane podejście zamiast instalacji globalnej na klastrze.

In [0]:
# Instalacja biblioteki emoji
%pip install emoji

In [0]:
import emoji
print(emoji.emojize('Databricks is :fire:'))

## Sekcja 4.5: Databricks Assistant (AI)

W roku 2025 praca z kodem jest wspomagana przez AI. Databricks posiada wbudowanego asystenta (**Databricks Assistant**), który jest świadomy kontekstu Twoich danych (zna schematy tabel w Unity Catalog!).

**Jak korzystać?**
1. Skrót **Cmd+I** (Mac) lub **Ctrl+I** (Windows) wewnątrz komórki.
2. Panel boczny "Assistant".

**Do czego służy?**
- **Generowanie kodu**: "Napisz zapytanie SQL, które policzy średnią sprzedaż po regionach z tabeli sales".
- **Wyjaśnianie kodu**: Zaznacz skomplikowany fragment i zapytaj "Explain this code".
- **Fixing errors**: Gdy komórka zwróci błąd, kliknij "Diagnose Error" – asystent wyjaśni przyczynę i zaproponuje poprawkę.
- **Transformacja**: "Przepisz ten kod z PySpark na SQL".

## Sekcja 5: Unity Catalog vs Hive Metastore

**Wprowadzenie teoretyczne:**

Databricks wspiera dwa systemy metadanych: legacy Hive Metastore oraz nowoczesny Unity Catalog. Unity Catalog jest zalecany dla wszystkich nowych projektów ze względu na zaawansowane funkcje governance i bezpieczeństwa.

**Kluczowe różnice:**

| Aspekt | Hive Metastore | Unity Catalog |
|--------|----------------|---------------|
| **Governance** | Ograniczone | Pełne: RBAC, masking, audit |
| **Namespace** | 2-poziomowy (db.table) | 3-poziomowy (catalog.schema.table) |
| **Cross-workspace** | Nie | Tak (shared metastore) |
| **Lineage** | Brak | End-to-end lineage |
| **Data Sharing** | Ograniczone | Delta Sharing protocol |
| **Isolation** | Workspace-level | Catalog-level |

**Dlaczego Unity Catalog?**
- Centralne zarządzanie dostępem dla wszystkich workspace'ów
- Automatyczny lineage dla audytu i compliance
- Fine-grained permissions (column-level, row-level)
- Integracja z zewnętrznymi systemami (Delta Sharing)

### Przykład 5.1: Namespace - Hive vs Unity Catalog

**Cel:** Porównanie składni dostępu do tabel

**Hive Metastore (legacy)**

- **Składnia**: `database.table`
- **Przykład**: `default.sales_data`
- **Ograniczenia**: Brak fine-grained permissions, brak lineage, workspace isolation

**Unity Catalog (nowoczesny)**

- **Składnia**: `catalog.schema.table`
- **Przykład**: `prod.gold.sales_summary`

**Zalety 3-poziomowego namespace:**
- Oddzielenie środowisk (dev/test/prod catalogs)
- Lepsze uprawnienia (grant na poziomie catalogu)
- Współdzielenie metastore między workspace'ami
- End-to-end lineage
- Fine-grained access control

### Przykład 5.2: Tworzenie tabeli w Unity Catalog

**Cel:** Demonstracja pełnej składni z 3-poziomowym namespace

In [0]:
# Utworzenie przykładowej tabeli w Unity Catalog
table_name = f"{CATALOG}.{BRONZE_SCHEMA}.lakehouse_demo"

# Dane demonstracyjne
demo_data = [
    (1, "Unity Catalog", "Enabled", "2024-01-15"),
    (2, "Delta Lake", "Enabled", "2024-01-15"),
    (3, "Photon Engine", "Enabled", "2024-01-15"),
    (4, "Hive Metastore", "Legacy", "2024-01-15")
]

In [0]:
# Definicja schematu
demo_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("feature", StringType(), False),
    StructField("status", StringType(), False),
    StructField("date", StringType(), False)
])

In [0]:
# Utworzenie DataFrame
demo_df = spark.createDataFrame(demo_data, demo_schema)
display(demo_df)

In [0]:
# Zapis jako Delta Table w Unity Catalog
demo_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_name)

In [0]:
# Weryfikacja
display(spark.table(table_name))

**Sukces!** Tabela została utworzona w Unity Catalog. Możemy teraz zweryfikować jej istnienie.

### 🎯 Zadanie: Sprawdź w UI

1. Kliknij **Catalog** w lewym menu bocznym.
2. Znajdź swój katalog (nazwa w zmiennej `CATALOG`, np. `ecommerce_platform_...`).
3. Rozwiń schemat `bronze` (lub inny zdefiniowany w `BRONZE_SCHEMA`).
4. Kliknij na tabelę `lakehouse_demo`.
5. Zobacz zakładki: **Sample Data** (podgląd) oraz **Lineage** (pochodzenie danych).

**Wyjaśnienie:**

Tabela została utworzona z pełnym 3-poziomowym namespace. W Unity Catalog każda tabela automatycznie:
- Jest zarządzana przez system governance
- Ma trackowany lineage
- Posiada przypisane uprawnienia na podstawie katalogu i schematu
- Jest dostępna w Catalog Explorer dla eksploracji

**Managed vs External Tables:**
Powyższa tabela to **Managed Table**. Databricks zarządza zarówno metadanymi, jak i plikami danych (w domyślnym storage katalogu/schematu). Usunięcie tabeli (`DROP TABLE`) usuwa również dane.

**External Table** powstaje, gdy podamy `LOCATION 'path'`. Wtedy `DROP TABLE` usuwa tylko metadane, a pliki pozostają w storage'u.

### Bonus: Delta Time Travel (Teaser)

Każda operacja na tabeli Delta jest rejestrowana w transaction logu. Dzięki temu możemy śledzić historię zmian i cofać się w czasie (Time Travel).

In [0]:
display(spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.lakehouse_demo"))

In [0]:
%sql
DESCRIBE HISTORY ecommerce_platform_trainer.bronze.lakehouse_demo

### Przykład 5.3: Eksploracja metadanych Unity Catalog

**Cel:** Wykorzystanie systemu informacyjnych schematów Unity Catalog

In [0]:
%sql
-- Unity Catalog udostępnia system.information_schema dla metadanych

-- Lista tabel w naszym schemacie
SELECT 
  table_catalog,
  table_schema,
  table_name,
  table_type
FROM system.information_schema.tables
WHERE table_catalog = 'ecommerce_platform_trainer'
  AND table_schema = 'bronze'
ORDER BY table_name

**Wyjaśnienie:**

Unity Catalog automatycznie utrzymuje `system.information_schema` - zbiór widoków SQL z metadanymi o wszystkich obiektach. To standardowe podejście zgodne z ANSI SQL, co ułatwia integrację z narzędziami BI i data governance.

**System Tables:**
Warto wiedzieć, że Unity Catalog udostępnia również tabele systemowe (wymaga włączenia przez admina):
- `system.billing.usage`: szczegółowe dane o kosztach (DBU)
- `system.access.audit`: logi audytowe (kto, co, kiedy)

## Porównanie PySpark vs SQL

**DataFrame API (PySpark):**

In [0]:
# Podejście PySpark - programatyczne DataFrame API

df_pyspark = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.lakehouse_demo")

In [0]:
result_pyspark = df_pyspark \
    .filter(F.col("status") == "Enabled") \
    .select("feature", "status", "date") \
    .orderBy("feature")

In [0]:
display(result_pyspark)

**SQL Equivalent:**

In [0]:
print(f"{CATALOG}.{BRONZE_SCHEMA}")

In [0]:
%sql

select * from 
ecommerce_platform_trainer.bronze.lakehouse_demo

### Parametryzacja z Databricks Widgets

Poniżej używamy mechanizmu **Widgets**, który pozwala na tworzenie interaktywnych kontrolek w notebooku. Dzięki temu możemy przekazywać parametry (np. nazwy tabel, daty) do kodu SQL i Python, co ułatwia budowanie uniwersalnych raportów.

In [0]:
# Parametryzacja z Databricks Widgets
# Ustawiamy wartości domyślne na podstawie zmiennych z 00_setup (jeśli dostępne)
# Dzięki temu SQL cells będą korzystać z tego samego katalogu co Python cells

default_catalog = CATALOG if 'CATALOG' in locals() else "ecommerce_platform_trainer"
default_schema = BRONZE_SCHEMA if 'BRONZE_SCHEMA' in locals() else "bronze"

dbutils.widgets.text("CATALOG", default_catalog)
dbutils.widgets.text("BRONZE_SCHEMA", default_schema)

In [0]:
%sql

SELECT 
  feature,
  status,
  date
FROM IDENTIFIER(:CATALOG || '.' || :BRONZE_SCHEMA || '.lakehouse_demo')
WHERE status = 'Enabled'
ORDER BY feature

**Porównanie:**
- **Wydajność**: Identyczna - oba podejścia kompilują się do tego samego Catalyst query plan
- **Kiedy używać PySpark**: 
  - Złożona logika biznesowa z UDF
  - Dynamiczne pipeline'y (parametryzacja, loops)
  - Integracja z bibliotekami Python (pandas, scikit-learn)
- **Kiedy używać SQL**: 
  - Proste transformacje i agregacje
  - Zespół z silnymi kompetencjami SQL
  - Migracja z tradycyjnych Data Warehouse
  - Lepsze wsparcie dla analityków biznesowych

## Walidacja i weryfikacja

### Checklist - Co powinieneś zrozumieć po tym notebooku:
- [x] Koncepcja Lakehouse i korzyści względem tradycyjnej architektury
- [x] Struktura Workspace: Sidebar, Compute, Catalog Explorer
- [x] Hierarchia Unity Catalog: Metastore → Catalog → Schema → Objects
- [x] Typy klastrów: All-Purpose vs Job, autoscaling, Photon
- [x] Magic commands: %sql, %python, %fs, %run, %md
- [x] Różnice między Hive Metastore (2-level) a Unity Catalog (3-level)
- [x] Tworzenie tabel w Unity Catalog z pełnym namespace
- [x] Dostęp do metadanych przez system.information_schema

## Troubleshooting

### Problem 1: "Catalog not found" lub "Schema not found"
**Objawy:**
- Błąd: `AnalysisException: [SCHEMA_NOT_FOUND]`
- Błąd: `AnalysisException: [CATALOG_NOT_FOUND]`

**Rozwiązanie:**
```python
# Sprawdź dostępne katalogi
spark.sql("SHOW CATALOGS").show()

# Sprawdź dostępne schematy
spark.sql(f"SHOW SCHEMAS IN {CATALOG}").show()

# Upewnij się, że uruchomiłeś %run ./00_setup
```

### Problem 2: "Permission denied" przy tworzeniu tabel
**Objawy:**
- Błąd: `PERMISSION_DENIED: User does not have CREATE TABLE on Schema`

**Rozwiązanie:**
- Skontaktuj się z administratorem workspace o nadanie uprawnień `CREATE TABLE`
- Sprawdź uprawnienia: `SHOW GRANTS ON SCHEMA catalog.schema`

### Problem 3: Klaster nie startuje lub jest zbyt wolny
**Objawy:**
- Klaster w stanie "Pending" przez długi czas
- Timeout przy starcie

**Rozwiązanie:**
- Sprawdź quota instancji w chmurze (Azure/AWS/GCP)
- Zmniejsz liczbę workerów lub wybierz mniejszy node type
- Wyłącz autoscaling dla testów

### Problem 4: Magic command %sql nie działa
**Objawy:**
- Błąd składni lub brak wyników

**Rozwiązanie:**
- Upewnij się, że `%sql` jest pierwszym elementem w komórce
- Sprawdź czy używasz zmiennych: `${CATALOG}` zamiast `{CATALOG}`
- Dla zmiennych Python użyj: `spark.sql(f"SELECT ... FROM {CATALOG}.{SCHEMA}.table")`

### Debugging tips:
- Użyj `explain()` na DataFrame aby zobaczyć plan wykonania
- Sprawdź logi klastra w Spark UI (zakładka "Cluster" → "Spark UI")
- Weryfikuj typy danych: `df.printSchema()`
- Dla problemów z wydajnością sprawdź liczbę partycji: `df.rdd.getNumPartitions()`

## Best Practices

### Architektura Lakehouse:
- Używaj Unity Catalog zamiast Hive Metastore dla nowych projektów
- Organizuj dane w logiczne katalogi (np. dev/test/prod)
- Stosuj naming convention dla schematów: bronze/silver/gold
- Wykorzystuj Delta Lake jako domyślny format tabel

### Zarządzanie Workspace:
- Organizuj notebooki w folderach według projektów lub zespołów
- Używaj Git Folders (Repos) dla integracji z Git i wersjonowania
- Dokumentuj notebooki z Markdown cells
- Stosuj `%run` dla współdzielenia kodu między notebookami

### Konfiguracja Klastrów:
- Development: małe klastry (2-4 workers), bez spot instances
- Production: autoscaling, spot instances dla oszczędności
- Włącz Photon Engine dla zapytań SQL/DataFrame
- Ustaw auto-termination (np. 30 min nieaktywności) dla klastrów All-Purpose
- Dla Jobs używaj Job Clusters (efemeryczne, optymalne koszty)

### Governance i Security:
- Stosuj per-user lub per-team izolację katalogów
- Używaj 3-poziomowego namespace: catalog.schema.table
- Przydzielaj uprawnienia na poziomie schematu, nie tabeli
- Monitoruj dostęp przez system.access.audit
- Włącz lineage dla compliance i debugowania

### Wydajność:
- Preferuj Delta Lake zamiast Parquet/CSV dla częstych zapytań
- Partycjonuj duże tabele według kluczy czasowych lub geograficznych
- Używaj Z-ORDER dla kolumn używanych w WHERE clauses
- Regularnie uruchamiaj OPTIMIZE i VACUUM na tabelach Delta

## Podsumowanie

### Co zostało osiągnięte:
- Poznanie koncepcji Lakehouse jako ewolucji Data Lake + Data Warehouse
- Eksploracja elementów platformy Databricks: Workspace, Compute, Catalog
- Zrozumienie hierarchii Unity Catalog: Metastore → Catalog → Schema → Objects
- Praktyka z magic commands: %sql, %python, %fs, %pip
- Porównanie Hive Metastore vs Unity Catalog
- Utworzenie pierwszej tabeli Delta w Unity Catalog z 3-poziomowym namespace

### Kluczowe wnioski:
1. **Lakehouse eliminuje duplikację danych**: Jedna kopia służy BI, ML i real-time analytics
2. **Unity Catalog to fundament governance**: 3-poziomowy namespace, fine-grained permissions, automatyczny lineage
3. **Klastry są elastyczne**: Autoscaling i spot instances redukują koszty, Photon przyspiesza zapytania
4. **Notebooki są potężne**: Mieszanie SQL/Python, magic commands, integracja z Git przez Git Folders
5. **Delta Lake jest domyślnym formatem**: ACID transactions, time travel, schema evolution

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | SQL |
|----------|---------|-----|
| Ustaw katalog | `spark.sql(f"USE CATALOG {CATALOG}")` | `USE CATALOG my_catalog` |
| Lista katalogów | `spark.sql("SHOW CATALOGS")` | `SHOW CATALOGS` |
| Lista schematów | `spark.sql("SHOW SCHEMAS")` | `SHOW SCHEMAS` |
| Utworzenie tabeli | `df.write.saveAsTable("cat.schema.table")` | `CREATE TABLE cat.schema.table AS SELECT ...` |
| Odczyt tabeli | `spark.table("cat.schema.table")` | `SELECT * FROM cat.schema.table` |
| Metadane | - | `SELECT * FROM system.information_schema.tables` |
| Instalacja lib | `%pip install package` | - |

### Następne kroki:
- **Kolejny notebook**: `02_data_import_exploration.ipynb` - Wczytywanie danych z różnych formatów (CSV, JSON, Parquet, Delta)
- **Warsztat praktyczny**: Po zakończeniu dema (notebooki 01-05) przejdziemy do `01_workspace_data_exploration_workshop.ipynb`
- **Materiały dodatkowe**: 
  - [Databricks Lakehouse Fundamentals](https://www.databricks.com/learn/lakehouse)
  - [Unity Catalog Documentation](https://docs.databricks.com/data-governance/unity-catalog/index.html)
  - [Delta Lake Guide](https://docs.delta.io/latest/index.html)